In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1 as ff1

ff1.Cache.enable_cache('cache')

In [73]:
import warnings
warnings.filterwarnings('ignore')

In [74]:
def get_empty_dataframe():
    return pd.DataFrame(
            columns=[
                'LapNumber',
                'LapTime',
                'Compound',
                'TyreLife',
                'TrackStatus',
                'Stint',
                'DistanceToDriverAhead',
                'DriverAhead',
                ])

In [75]:
class NoTelemetryException(Exception):
    pass

def get_telemetry_at_start_of_lap(lap, telemetry):
    mask = (telemetry['Date'] > lap['LapStartDate']) & (telemetry['Date'] <= lap['LapStartDate'] + pd.Timedelta(seconds=1))
    rows = telemetry.loc[mask]
    if rows.empty:
        raise NoTelemetryException("No telemetry data found for lap " + str(lap['LapNumber']) + " of " + str(lap['Driver']) + " at " + str(lap['LapStartDate']))
    row = rows.iloc[0]
    telemetryInfo = row[['DriverAhead', 'DistanceToDriverAhead']]
    lapInfo = lap[['LapNumber', 'LapTime', 'Compound', 'TyreLife', 'Stint', 'TrackStatus']]
    telemetryInfo.rename("Telemetry", inplace=True)
    lapInfo.rename("Lap", inplace=True)
    merge = pd.concat([telemetryInfo, lapInfo])
    return merge

In [90]:
class NoLapException(Exception):
    pass

def get_laps_of_driver(driver, laps):
    driver_laps = laps.pick_driver(driver)
    if len(driver_laps['DriverNumber']) == 0:
        raise NoLapException("No laps for driver " + driver)
    driver_laps_telemetry = driver_laps.get_car_data()
    if len(driver_laps_telemetry) == 0:
        raise NoLapException("No telemetry for driver " + driver)
    try:
        driver_laps_telemetry = driver_laps_telemetry.add_driver_ahead()
        transformed_laps = []
        for index, row in driver_laps.iterrows():
            try:
                transformed_laps.append(get_telemetry_at_start_of_lap(row, driver_laps_telemetry))
            except NoTelemetryException as e:
                print(e)
    except ValueError as e:
        print("error : ", e)
        print(driver_laps_telemetry)
        return get_empty_dataframe()
    return pd.DataFrame(transformed_laps)

In [77]:
import os

def get_season_data(year, save_all_races=True):
    schedule = ff1.get_event_schedule(year, include_testing=False)
    
    path = 'data/' + str(year)
    # Create a directory for the year if it doesn't exist
    if not os.path.exists(path):
        os.makedirs(path)

    df_season = get_empty_dataframe()
    for index, event in schedule.iterrows():
        print("Processing race round - ", event['RoundNumber'])
        race = event.get_race()
        race.load()
        df_event = get_empty_dataframe()
        for driver in race.drivers:
            print("     Processing driver - ", driver)
            try:
                df_event = pd.concat([df_event, get_laps_of_driver(driver, race.laps)], axis=0)
            except NoLapException as e:
                print(e)
        
        if save_all_races:
            # Save it to a csv file
            df_event.to_csv(path + '/' + event.EventName.replace(' ', '_').lower() + '.csv', index=False)
        df_season = pd.concat([df_season, df_event], axis=0)

    # Save the data for the whole season
    df_season.to_csv(path + '/season.csv', index=False)
    return df_season

In [99]:
year = 2018
schedule = ff1.get_event_schedule(year, include_testing=False)

path = 'data/' + str(year)
# Create a directory for the year if it doesn't exist
if not os.path.exists(path):
    os.makedirs(path)

df_season = get_empty_dataframe()

event = schedule.get_event_by_round(2)
race = event.get_race()
race.load(laps=True, telemetry=True, weather=True, messages=True)


core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	No cached data found for car_data. Loading data...
api            INFO 	Fetching car data...
core        WARNING 	Failed to load telemetry data!
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '77', '44', '10', '20', '27', '14', '2', '9', '31', '55', '16', '8', '18', '35', '11', '28', '7', '33', '3']


In [100]:
df_event = get_empty_dataframe()
for driver in race.drivers:
    print("     Processing driver - ", driver)
    try:
        df_event = pd.concat([df_event, get_laps_of_driver(driver, race.laps)], axis=0)
    except NoLapException as e:
        print(e)


# Save it to a csv file
df_event.to_csv(path + '/' + event.EventName.replace(' ', '_').lower() + '.csv', index=False)
df_season = pd.concat([df_season, df_event], axis=0)

# Save the data for the whole season
df_season.to_csv(path + '/season.csv', index=False)

     Processing driver -  5


DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`

In [93]:
get_season_data(2018)

core           INFO 	Loading data for Australian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info


Processing race round -  1


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	No cached data found for position_data. Loading data...
api            INFO 	Fetching position data...
core        WARNING 	Failed to load telemetry data!
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '44', '7', '3', '14', '33', '27', '77', '2', '55', '11', '31', '16', '18', '28', '8', '20', '10', '9', '35']


     Processing driver -  5


DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`

In [80]:
for year in range(2018, 2022):
    get_season_data(year)

core           INFO 	Loading data for Australian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info


Processing race round -  1


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	No cached data found for position_data. Loading data...
api            INFO 	Fetching position data...
core        WARNING 	Failed to load telemetry data!
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '44', '7', '3', '14', '33', '27', '77', '2', '55', '11', '31', '16', '18', '28', '8', '20', '10', '9', '35']


     Processing driver -  5


DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`